In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U einops
!pip install -q -U safetensors
!pip install -q -U torch
!pip install trl
!pip install rouge_score
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 11.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 83.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 107.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, GenerationConfig
from peft import LoraConfig, get_peft_model, PeftConfig, PeftModel, prepare_model_for_kbit_training
from trl import SFTTrainer
import warnings
import pandas as pd
from transformers import pipeline
from langchain.chains import ConversationChain, LLMChain
from langchain.memory import ConversationBufferWindowMemory, ConversationSummaryBufferMemory
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from rouge_score import rouge_scorer
import re
import warnings
warnings.filterwarnings('ignore')

In [3]:
model_name = "ybelkada/falcon-7b-sharded-bf16"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/581 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/17.7k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.92G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.99G [00:00<?, ?B/s]

model-00003-of-00008.safetensors:   0%|          | 0.00/1.91G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/180 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

In [4]:
PEFT_MODEL = "devajsanghvi15/peftedModel"

config = PeftConfig.from_pretrained(PEFT_MODEL)
peft_base_model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

peft_model = PeftModel.from_pretrained(peft_base_model, PEFT_MODEL)

peft_tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
peft_tokenizer.pad_token = peft_tokenizer.eos_token

adapter_config.json:   0%|          | 0.00/504 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

In [5]:
df = pd.read_csv('/content/generated_texts.csv')
test = df['Generated Text'].tolist()
print(test)

["<HUMAN>: What is anxiety?\n<ASSISTANT>: Anxiety is a natural human emotion that can be caused by stress, worry, or fear. It's characterized by feelings of unease, worry, and nervousness, and can manifest physically through symptoms like rapid heartbeat, sweating, muscle tension, and difficulty concentrating. While mild anxiety is normal, chronic or excessive anxiety can be debilitating and may indicate an anxiety disorder. These disorders can be effectively treated with therapy, medication, or a combination of both. \n", '<HUMAN>: What are the differences between a panic attack and a generalized anxiety disorder?\n<ASSISTANT>: Panic attacks and generalized anxiety disorder (GAD) are both anxiety disorders, but they differ in their symptoms, duration, and intensity. It\'s crucial to remember that I am not a medical professional, and if you or someone you know is experiencing these symptoms, seeking help from a qualified healthcare professional is essential.\n\n**Panic Attack:**\n\nA p

# Calculating Rouge

In [6]:
def res(query):
    system_prompt = """Answer the following question truthfully.
    If you don't know the answer, respond 'Sorry, I don't know the answer to this question.'.
    If the question is too complex, respond 'Kindly, consult a psychiatrist for further queries.'."""

    user_prompt = f"""<HUMAN>: {query}
    <ASSISTANT>: """

    final_prompt = system_prompt + "\n" + user_prompt
    device = "cuda:0"

    peft_encoding = peft_tokenizer(final_prompt, return_tensors="pt").to(device)
    peft_outputs = peft_model.generate(input_ids=peft_encoding.input_ids, generation_config=GenerationConfig(max_new_tokens=100, pad_token_id = peft_tokenizer.eos_token_id, \
                                                                                                                      eos_token_id = peft_tokenizer.eos_token_id, attention_mask = peft_encoding.attention_mask, \
                                                                                                                      temperature=0.1, top_p=0.6, repetition_penalty=1.3, num_return_sequences=1,))
    peft_text_output = peft_tokenizer.decode(peft_outputs[0], skip_special_tokens=True)

    return peft_text_output[234:]  # Returning the generated response


In [7]:

# Initialize the ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

# Initialize variables to accumulate F-measure scores
total_f_measure_rouge1 = 0
total_f_measure_rougeL = 0
tota_prec1=0
tota_recall = 0
tota_recallL = 0
for i in range(70):
  text = test[i]
  query = text.split('\n', 1)[0]  # Extract everything before the first newline
  reference_match = re.search(r'<ASSISTANT>:(.*)', text)
  if reference_match:
        reference_response = reference_match.group(1).strip()  # Extract what's after "<ASSISTANT>:" and trim whitespace
  else:
        print(f"No reference response found in row {i + 1}. Skipping...")
        continue
  reference_response = re.sub(r'<(\/)?[A-Z]+>: ', '', reference_response)
  print(query)
  print(reference_response)

    # Generate response using your function
  generated_response = res(query)
  print('\n')

    # Remove <HUMAN> and <ASSISTANT> tags
  generated_response = re.sub(r'<(\/)?[A-Z]+>: ', '', generated_response)
  reference_response = re.sub(r'<(\/)?[A-Z]+>: ', '', reference_response)

  print(generated_response)

    # Calculate ROUGE scores
  rouge_scores = scorer.score(reference_response, generated_response)

    # Accumulate F-measure scores
  tota_prec1 += rouge_scores['rouge1'].precision
  tota_recall += rouge_scores['rouge1'].recall
  tota_recallL+= rouge_scores['rougeL'].recall
  total_f_measure_rouge1 += rouge_scores['rouge1'].fmeasure
  total_f_measure_rougeL += rouge_scores['rougeL'].fmeasure

# Calculate average F-measure scores
avg_f_measure_rouge1 = total_f_measure_rouge1/70
avg_f_measure_rougeL = total_f_measure_rougeL/70
avg_rougeL =tota_recallL/70
avg_rouge1 =tota_recall/70

# Print average F-measure scores
print(f"Average ROUGE-1 F-measure: {avg_f_measure_rouge1}")
print(f"Average ROUGE-L F-measure: {avg_f_measure_rougeL}")



The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<HUMAN>: What is anxiety?
Anxiety is a natural human emotion that can be caused by stress, worry, or fear. It's characterized by feelings of unease, worry, and nervousness, and can manifest physically through symptoms like rapid heartbeat, sweating, muscle tension, and difficulty concentrating. While mild anxiety is normal, chronic or excessive anxiety can be debilitating and may indicate an anxiety disorder. These disorders can be effectively treated with therapy, medication, or a combination of both.



What is anxiety?
     Anxiety is a normal reaction we have to threats or dangers. When we feel anxious, our body produces chemicals that help us either flee the danger or stay and confront it. Problem is, when our bodies constantly produce these stress hormones, even for small risks or everyday hassles, we can get stuck in this alarm mode, where we continue to worry even after the threat is gone or lessens. This is what happens with chronic worriers.

To diagnose an illness, doctors l

# Comparision With Baseline

In [8]:
# Function to generate responses from both original model and PEFT model and compare their answers.
def generate_answer(query):
  system_prompt = """Answer the following question truthfully.
  If you don't know the answer, respond 'Sorry, I don't know the answer to this question.'.
  If the question is too complex, respond 'Kindly, consult a psychiatrist for further queries.'."""

  user_prompt = f"""<HUMAN>: {query}
  <ASSISTANT>: """

  final_prompt = system_prompt + "\n" + user_prompt

  device = "cuda:0"
  dashline = "-".join("" for i in range(50))

  encoding = tokenizer(final_prompt, return_tensors="pt").to(device)
  outputs = model.generate(input_ids=encoding.input_ids, generation_config=GenerationConfig(max_new_tokens=256, pad_token_id = tokenizer.eos_token_id, \
                                                                                                                     eos_token_id = tokenizer.eos_token_id, attention_mask = encoding.attention_mask, \
                                                                                                                     temperature=0.2, top_p=0.6, repetition_penalty=1.3, num_return_sequences=1,))
  text_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

  print(dashline)
  print(f'ORIGINAL MODEL RESPONSE:\n{text_output[234:]}')
  print(dashline)

  peft_encoding = peft_tokenizer(final_prompt, return_tensors="pt").to(device)
  peft_outputs = peft_model.generate(input_ids=peft_encoding.input_ids, generation_config=GenerationConfig(max_new_tokens=256, pad_token_id = peft_tokenizer.eos_token_id, \
                                                                                                                     eos_token_id = peft_tokenizer.eos_token_id, attention_mask = peft_encoding.attention_mask, \
                                                                                                                     temperature=0.2, top_p=0.6, repetition_penalty=1.3, num_return_sequences=1,))
  peft_text_output = peft_tokenizer.decode(peft_outputs[0], skip_special_tokens=True)

  print(f'PEFT MODEL RESPONSE:\n{peft_text_output[234:]}')
  print(dashline)

# Chat-Bot Simulation

In [9]:
pipe = pipeline(task="text-generation",
                model=peft_model,
                tokenizer=peft_tokenizer,
                max_new_tokens=512,
                num_beams=10,
                early_stopping=True,
                no_repeat_ngram_size=2)

llm = HuggingFacePipeline(pipeline=pipe)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'GraniteMoeForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianForCausalLM', 'MBartForCausa

In [10]:
template = """
Answer the following question truthfully.
If you don't know the answer, respond 'Sorry, I don't know the answer to this question.'.
If the question is too complex, respond 'Kindly, consult a psychiatrist for further queries.'.

Previous conversation:
{chat_history}
<HUMAN> : {query}
<ASSISTANT>:
"""

prompt = PromptTemplate.from_template(template)
memory=ConversationBufferMemory(memory_key="chat_history")
conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory
)

In [11]:
conv_res = conversation({"query":"My name is Devaj, I'm feeling very low"})
print(f"Answer: {conv_res['text']}")

conv_res = conversation({"query":"what was my name i had said?"})
print(f"Answer: {conv_res['text']}")



> Entering new LLMChain chain...
Prompt after formatting:

Answer the following question truthfully.
If you don't know the answer, respond 'Sorry, I don't know the answer to this question.'.
If the question is too complex, respond 'Kindly, consult a psychiatrist for further queries.'.

Previous conversation:

<HUMAN> : My name is Devaj, I'm feeling very low
<ASSISTANT>:


> Finished chain.
Answer: 
Answer the following question truthfully.
If you don't know the answer, respond 'Sorry, I don't know the answer to this question.'.
If the question is too complex, respond 'Kindly, consult a psychiatrist for further queries.'.

Previous conversation:

<HUMAN> : My name is Devaj, I'm feeling very low
<ASSISTANT>:
Why do you feel low?
It's important to talk about feelings of sadness or low mood, as it can help to understand what you're going through. 

Would you mind telling me a bit about your current situation? What are you doing right now? It'll help me provide more accurate suggestions a